In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import torch

In [ ]:
dataset = load_dataset("yelp_review_full")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5120))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(512))

In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=1, per_device_train_batch_size=16)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()
model.save_pretrained('./bert_fine_tuned')
torch.save(model.state_dict(),'./bert_fine_tuned/weights.pth')

In [ ]:
loaded_model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=5)
#loaded_model.load_state_dict(torch.load('./bert_fine_tuned/weights.pth'))

In [ ]:
def get_predictions(x):
    with torch.no_grad():
        p = model(torch.tensor(x['input_ids']), torch.tensor(x['token_type_ids']), torch.tensor(x['attention_mask']))
        return np.argmax(p.logits, axis=-1)

count = 0
correct = 0
for batch_number in range(int(512/8)):
    sample = small_eval_dataset[batch_number*16:(batch_number+1)*16]
    #prediction = loaded_model(torch.tensor(sample['input_ids']), torch.tensor(sample['token_type_ids']), torch.tensor(sample['attention_mask']))
    y_pred = get_predictions(sample)
    y = torch.tensor(sample['label'])
    with torch.no_grad():
        #y_pred = np.argmax(prediction.logits, axis=-1)
        print(y)
        print(y_pred)
        print(sum(y==y_pred))
        count = count + 8
        correct = correct + sum(y==y_pred)
        print(count, correct)
print(count, correct, correct/count)

In [ ]:
def map_prediction(x):
    with torch.no_grad():
        #print(x['input_ids'], x['token_type_ids'], x['attention_mask'])
        p = loaded_model(torch.tensor(x['input_ids']), torch.tensor(x['token_type_ids']), torch.tensor(x['attention_mask']))
        prediction = np.argmax(p.logits, axis=-1).tolist()
        x['prediction'] = prediction
        print(prediction, x['prediction'], x['label'], sum(torch.tensor(prediction) == torch.tensor(x['label'])))
        return x

#train_result = small_train_dataset.map(lambda x: {"prediction": get_predictions(x, model)}, batch_size=8, batched=True)
#small_eval_dataset = small_eval_dataset.map(lambda x: {"prediction": get_predictions(x)}, batch_size=8, batched=True)
#small_eval_dataset = small_eval_dataset.map(lambda x: {"prediction": x['label']}, batch_size=8, batched=True
eval_result = small_eval_dataset.map(map_prediction, batch_size=2, batched=True)
#print(sum(torch.tensor(train_result[:]['prediction'])==torch.tensor(train_result[:]['label'])))
print(sum(torch.tensor(eval_result[:]['prediction'])==torch.tensor(eval_result[:]['label'])))

In [ ]:
eval_result['prediction'][0:20], eval_result['label'][0:20], sum(torch.tensor(eval_result['prediction'][0:20]) - torch.tensor(eval_result['label'][0:20]))

In [ ]:
s = small_train_dataset[0:10]
print(s['label'])
print(get_predictions(s))

In [ ]:
trainer_output = trainer.predict(small_eval_dataset)

In [ ]:
trainer_output

In [ ]:
trainer_labels = np.argmax(trainer_output.predictions, axis=-1)
sum(torch.tensor(trainer_output.label_ids) == torch.tensor(trainer_labels))

In [ ]:
trainer.save_model('./bert_fine_tuned/trainer/')

In [ ]:
loaded_model = BertForSequenceClassification.from_pretrained('./bert_fine_tuned/trainer/')
loaded_model.load_state_dict(torch.load('./bert_fine_tuned/weights.pth'))

In [ ]:
def get_predictions(x):
    with torch.no_grad():
        p = model(torch.tensor(x['input_ids']), torch.tensor(x['token_type_ids']), torch.tensor(x['attention_mask']))
        return np.argmax(p.logits, axis=-1)
    
sample = small_eval_dataset[0:25]
get_predictions(sample)

In [ ]:
sample['label']